### Install Required Packages

We begin by installing and upgrading the required libraries: `openai`, `gradio`, `textblob`, and `matplotlib`. These are necessary for generating responses, building the UI, analyzing sentiment, and logging conversations.


In [ ]:
# Install required packages
!pip install --upgrade --quiet openai==0.28 gradio textblob matplotlib
!python -m textblob.download_corpora


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.9/322.9 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 111.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.5/11.5 MB 83.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.8 MB/s eta 0:00:00
[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk

### Import Libraries & Set API Key

We import the required Python libraries and load the OpenAI API key securely. Make sure your OpenAI key is stored in a safe location.


In [ ]:
# Import required libraries
import openai
import gradio as gr
from textblob import TextBlob
import pandas as pd
import matplotlib.pyplot as plt
import os

# Load OpenAI API key securely from Colab secrets
from google.colab import userdata
openai.api_key = userdata.get("openai")

# Create directory for logging chat history
os.makedirs("logs", exist_ok=True)
LOG_FILE = "logs/chat_log.csv"

### Analyze Sentiment of User Input

This function uses `TextBlob` to classify user messages into Positive, Neutral, or Negative sentiments.


In [ ]:
# Define function to analyze sentiment of user input
def get_sentiment(text):
    polarity = TextBlob(text).sentiment.polarity
    if polarity > 0.2:
        return "Positive"
    elif polarity < -0.2:
        return "Negative"
    else:
        return "Neutral"

### Chatbot Response Function

This function handles the conversation flow:
- Builds the conversation history
- Sends a prompt to the OpenAI API
- Analyzes sentiment of the user input
- Logs the full conversation to `chat_log.csv`


In [ ]:
# Define chatbot function to generate AI response, analyze sentiment, and log the interaction
def chatbot_response(prompt, chat_history):
    try:
        messages = [{"role": "system", "content": "You are a psychologist specializing in relationships."}]
        for user_input, bot_reply in chat_history:
            messages.append({"role": "user", "content": user_input})
            messages.append({"role": "assistant", "content": bot_reply})
        messages.append({"role": "user", "content": prompt})

        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        bot_reply = response.choices[0].message["content"]
        sentiment = get_sentiment(prompt)

        log_entry = pd.DataFrame([{
            "User Message": prompt,
            "Sentiment": sentiment,
            "Bot Response": bot_reply
        }])

        log_entry.to_csv(LOG_FILE, mode='a', header=not os.path.exists(LOG_FILE), index=False)

        chat_history.append((prompt, bot_reply))
        return chat_history, ""
    except Exception as e:
        return chat_history, f"API Error: {str(e)}"

### Clear Chat Function

Used to reset the chat history when the user presses the "Clear Chat" button.


In [ ]:
# Define function to clear chat history
def clear_chat():
    return [], []

### Launch the Chatbot Interface with Gradio

We define a user-friendly interface with:
- Chat display area
- Text input box
- Send and Clear buttons


In [ ]:
# Build Gradio interface for the chatbot
with gr.Blocks() as app:
    gr.Markdown("## Relationship Advice Chatbot")
    gr.Markdown("Ask anything about relationships and get expert guidance from your AI psychologist.")

    chatbot = gr.Chatbot(label="Your Relationship Advisor")
    msg = gr.Textbox(placeholder="Type your message here...", label="Your Message")
    send = gr.Button("Send")
    clear = gr.Button("Clear Chat")
    history = gr.State([])

    send.click(fn=chatbot_response, inputs=[msg, history], outputs=[chatbot, msg])
    clear.click(fn=clear_chat, outputs=[chatbot, history])

app.launch(debug=True)


<ipython-input-6-7b311a4af9dd>:5: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Your Relationship Advisor")


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://7c393d2f0b5ccf7b5c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
